<a href="https://colab.research.google.com/github/jjangmo91/Cervus-nippon_Anmado-Is./blob/main/Cervus-nippon_reconstruct_GF-SG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## GF-SG 방법을 사용하여 데이터를 다운받습니다. 자세한 내용은 아래에서 소개하는 논문을 읽어보시기 바랍니다.

Yang Chen, Ruyin Cao, Jin Chen, Licong Liu, Bunkei Matsushita,
A practical approach to reconstruct high-quality Landsat NDVI time-series data by gap filling and the Savitzky–Golay filter,
ISPRS Journal of Photogrammetry and Remote Sensing,
Volume 180,
2021,
Pages 174-190,
ISSN 0924-2716,
https://doi.org/10.1016/j.isprsjprs.2021.08.015.
(https://www.sciencedirect.com/science/article/pii/S0924271621002215)

The GEE code is freely available at https://code.earthengine.google.com/3a883c9e84ad119045bcb88e4de77b47?noload=true.


## GS-SG 방법은 크게 다음과 같은 단계로 작업이 이루어집니다.

1단계: Landsat, MODIS 데이터를 전처리(구름 마스킹, 품질 필터링, NDVI 계산)합니다.

2단계: MODIS NDVI 시계열에서 결측값을 선형 보간을 통해 채웁니다.

3단계: Savitzky-Golay 필터를 적용하여 보간된 MODIS 시계열 데이터를 스무딩합니다.
(각 데이터 구간에 다항식을 적합시켜 스무딩 진행)

4단계: MODIS NDVI데이터를 30m 해상도로 리샘플링하고(bicubic), Landsat 데이터와 결합합니다.

5단계: 선형 회귀 기반 형상 보정(shape Correction)을 통해 MODIS 데이터를 Landsat 시계열과 일치시키는 과정을 수행합니다.

6단계: 결합된 데이터를 사용해 연속적이고 부드러운 NDVI 시계열을 고해상도로 재구성합니다.

이 방법은 MODIS와 Landsat 데이터를 결합하여, 시간적으로 밀집되고, 공간적으로 고해상도의 NDVI 시계열 데이터를 재구성합니다.

아래 Javascript로 구성된 코드는 Google Eearth Engine Code Editor에서 실행하여 필요한 자료를 다운로드 받을 수 있습니다.

여기서는 기간과 안마도가 포함되는 공간(Landsat WRS 2 Descending Path Row)으로 수정하였습니다.

In [ ]:
var mod_sr = ee.ImageCollection("MODIS/006/MOD09Q1"),
    landsat_sr = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2"),

//---------------------------------------------------------------------------
//the GEE JavaScript code for Gap Filling and Savitzky–Golay filtering method (GF-SG)
//GF_SG is used for reconstructing high-quality Landsat NDVI time series data of 8-day intervals.
//GF-SG first fills missing values in the original Landsat NDVI time-series data by integrating the MODIS NDVI time-series data and cloud-free Landsat observations.
//A weighted Savitzky-Golay filter is used to remove the residual noise in the synthesized time series.
//Using MODIS surface reflectance time series data and Landsat surface reflectance time series data during one period.
//The program can be used for whole Landsat(TM/ETM/OLI) scene. When processing large areas, it is recommended to apply GF-SG on the Landsat scene one by one.
//the code of Interpolation_v1 is available on: https://code.earthengine.google.com/7eb9f43fd3da7fdfda5bfe432a5316e7?noload=true
//Note: this version has calculted the NDVI and determined the parameters of SG filter. Users can calculate other vegetation indexes and choose optimal parameters.
//      the discussion of SG parameters can be found in the following paper:
//      Chen, J., Jönsson, P., et al.(2004).A simple method for reconstructing a high-quality NDVI time-series data set based on the Savitzky-Golay filter. Remote Sensing of Environment
//      Cao, R.Y., Chen, Y., et al.(2018).A simple method to improve the quality of NDVI time-series data by integrating spatiotemporal information with the Savitzky-Golay filter. Remote Sensing of Environment
//Developed by Yang Chen, Email: yangchen.940407@gmail.com or yangchen0407@mail.bnu.edu.cn
//Update history:
//              06/2021, add cloud shadows as the invalid value in MODIS data
//              08/2021, 1.update the band name of Landsat 8 SR (B4 -> SR_B4;B5 -> SR_B5;pixel_qa -> QA_PIXEL)
//                       and geneneration of Landsat 8 cloud mask
//                       2.Users can choose to save a sub area or a period of time series from output data (L348,L366)
//The latest version and information of GF-SG can be found at
//            "https://github.com/ChenY04/GEE.git"
//please cite: Chen, Y., Cao R.Y., Chen, J., Liu, L.C., and Matsushita, B. A practical approach to reconstruct high-quality Landsat NDVI time-series data by gap
//filling and the Savitzky–Golay filter. ISPRS Journal of Photogrammetry and Remote Sensing.
//---------------------------------------------------------------------------

/*Extract bitpacked flags
 * image - the band to extract the bits from.
 * start - the lowest bit to extract
 * end - the highest bit to extract*/
var getQABits = function(image, start, end) {
    var pattern = 0;
    for (var i = start; i <= end; i++) {
       pattern += 1 << i;
    }
    // Return a single band image of the extracted QA bits
    return image.bitwiseAnd(pattern).rightShift(start);
};

//main procedure
var begin_date = ee.String('2014-01-01');  //Start date of processing time
var end_date = ee.String('2014-12-31');  //End date of processing time
var path = ee.Number(116);
var row = ee.Number(35);  //target Landsat scene (e.g.: 93/84)

//Calculate Landsat NDVI time series data
var landsat_ndvi_mask = landsat_sr.filterDate(begin_date, end_date)
   .filter(ee.Filter.eq('WRS_PATH', path))
   .filter(ee.Filter.eq('WRS_ROW', row))
   .map(function(img) {
   var img_sub = img;
   var ndvi = img_sub.normalizedDifference(['SR_B5','SR_B4']).rename('L_ndvi');  //calculate Landsat 8 NDVI
   //var ndvi = img_sub.normalizedDifference(['B4', 'B3']).rename('L_ndvi');  //calculate Landsat 5/7 NDVI
   var Q = img_sub.select('QA_PIXEL');
   //var Q = img_sub.select('pixel_qa');  //qa band for Landsat 5/7
   var cloud2BitMask = (1 << 2);
   var cloud3BitMask = (1 << 3);
   var cloud4BitMask = (1 << 4);
   var mask = Q.bitwiseAnd(cloud2BitMask).eq(0).and(Q.bitwiseAnd(cloud3BitMask).eq(0))
   .and(Q.bitwiseAnd(cloud4BitMask).eq(0)).rename('L_cmask');  //generate Landsat 8 cloud mask from pixel_qa band (0:clear;1: cloud/shadow)
   /*//generate Landsat 5/7 cloud mask
   var cloud3BitMask = (1 << 3);
   var cloud5BitMask = (1 << 5);
   var mask = Q.bitwiseAnd(cloud3BitMask).eq(0).and(Q.bitwiseAnd(cloud5BitMask).eq(0)).rename('L_cmask');
   */
   return ndvi.addBands(mask).copyProperties(img_sub, img_sub.propertyNames());});

var landsat_ndvi = landsat_ndvi_mask.select('L_ndvi');
var landsat_cmask = landsat_ndvi_mask.select('L_cmask');

//Show the location of target area
print(landsat_ndvi);
var mod_shp = landsat_ndvi.first().geometry();
Map.addLayer(mod_shp);
Map.centerObject(mod_shp);


//Calculate MODIS NDVI time series data
var mod_ndvi = mod_sr.filterDate(begin_date, end_date).filterBounds(mod_shp)
   .map(function(image) {
   var image0 = image;
   var img_sub = image0.clip(mod_shp);
   var ndvi = img_sub.normalizedDifference(['sur_refl_b02', 'sur_refl_b01']);  //calculate Landsat NDVI
   var Q = img_sub.select('State');
   var masknum = getQABits(Q, 0, 2);
   var mask = masknum.eq(ee.Image(0)).or(masknum.eq(ee.Image(3)));
   var ndvi_masked = ndvi.updateMask(mask);   //generate MODIS cloud mask from state band (0:clear;1: cloud/shadow/mixed)
   return ndvi_masked.rename(['MOD_NDVI']).copyProperties(img_sub, img_sub.propertyNames());});

//fill the gaps in the original MODIS NDVI time series by linear interpolation
var interl_m = require('users/Yang_Chen/GF-SG:Interpolation_v1');
var frame  = 8*4;
var nodata = -9999;
var mod_ndvi_interp = interl_m.linearInterp(mod_ndvi, frame, nodata);
var mod_ndvi_interp0 = mod_ndvi_interp.select(['MOD_NDVI_INTER']);

//Smooth the MODIS interpolation time series by the Savitzky–Golay filter
var sg_filter = require('users/Yang_Chen/GF-SG:SG_filter_v1');
var list_trend_sgCoeff = ee.List([-0.076923102,-1.4901161e-008,0.062937059,0.11188812,0.14685316,0.16783218,
0.17482519,0.16783218,0.14685316,0.11188812,0.062937059,-1.4901161e-008,-0.076923102]);  //suggested trend parameter: (6,6,0,2), Users can choose other optimal parameters.
var list_sg_coeff = ee.List([-0.090909064,0.060606077,0.16883118,0.23376624,0.25541127,0.23376624,
0.16883118,0.060606077,-0.090909064]);  //suggested parameter of filtering: (4,4,0,3), Users can choose other optimal parameters.
var mod_ndvi_sg = sg_filter.sg_filter_chen(mod_ndvi_interp0,list_trend_sgCoeff,list_sg_coeff);

//Resample MODIS NDVI images to 30m by using the bicubic interpolation method
var L_ref = landsat_ndvi.first();
var proj = ee.Image(L_ref).select('L_ndvi').projection().crs();
var mod_ndvi_sg30 = mod_ndvi_sg.map(function(img){
	return img.multiply(1.023).subtract(0.013).float().resample('bicubic')
	.copyProperties(img, img.propertyNames());});

var combine_ndvi = landsat_ndvi.merge(mod_ndvi_sg30);
var combine_ndvi0 = landsat_cmask.merge(mod_ndvi_sg30);

//Get the date of each Landsat image and each MODIS image
var startDate = ee.Date(begin_date);
var l_size = ee.Number(landsat_ndvi.size());
var m_size = ee.Number(mod_ndvi_sg30.size());
var combinendvi_list = combine_ndvi.toList(l_size.add(m_size));
var combinendvi_list0 = combine_ndvi0.toList(l_size.add(m_size));
var landsatndvi_list = combinendvi_list.slice(0,l_size);
var landsatmask_list = combinendvi_list0.slice(0,l_size);
var modndvi_list = combinendvi_list.slice(l_size,l_size.add(m_size));
var list_process = ee.List.sequence(0,l_size.subtract(1));
var ldate = list_process.map(function(i){
    i = ee.Number(i);
    var one = landsatndvi_list.get(i);
    var date = ee.Image(one).date();
    var relDoy = date.difference(startDate, 'day');
    return relDoy.toInt();});
var list_process = ee.List.sequence(0,m_size.subtract(1));
var mdate = list_process.map(function(i){
    i = ee.Number(i);
    var one = modndvi_list.get(i);
    var date = ee.Image(one).date();
    var relDoy = date.difference(startDate, 'day');
    return relDoy;});


//Get the date with both modis and landsat
var mdate0 = ee.Array(mdate);
var list_process = ee.List.sequence(0,l_size.subtract(1));
var lm_loc = list_process.map(function(i){
    i = ee.Number(i);
    var onenum = ee.Number(ldate.get(i));
    var diff = mdate0.subtract(onenum).abs().toList();
    var minloc = diff.indexOf(diff.reduce(ee.Reducer.min()));
    return minloc;});

//extract corresponding MODIS images of each Landsat image
var list_loc_old = ee.List.repeat(1,m_size);
var replace = function(current,previous){
  previous = ee.List(previous);
  var i = ee.Number(current);
  var oneloc = ee.Number(lm_loc.get(i));
  var list_loc0 = previous.set(oneloc,0);
  return list_loc0;
};
var list_loc0 = ee.List.sequence(0,l_size.subtract(1)).iterate(replace,list_loc_old);
var list_loc = ee.List(list_loc0);
var list_process = ee.List.sequence(0,l_size.subtract(1));
var modndviPair_list = list_process.map(function(i){
    i = ee.Number(i);
    var onenum = ee.Number(lm_loc.get(i));
    var onelist = modndvi_list.get(onenum);
    return onelist;});
var modndviPair = ee.ImageCollection(modndviPair_list);

//Generate adjusted MODIS NDVI time series (series after shape matching and correction)
var list_process = ee.List.sequence(0,l_size.subtract(1));
var landsatNdviPair_list = list_process.map(function(i){
  i = ee.Number(i);
  var landsatone = landsatndvi_list.get(i);
  var landsatone_img = ee.Image(landsatone);
  var maskone = landsatmask_list.get(i);
  var maskone_img = ee.Image(maskone);
  var landsatone_masked = landsatone_img.updateMask(maskone_img);  //mask cloudy pixels in Landsat out
  return landsatone_masked;});
var landsatNdviPair = ee.ImageCollection(landsatNdviPair_list);


var W0 = ee.List.repeat(1,31); var W = ee.List.repeat(W0 ,31);  //the size of local window for searching the neighboring similar MODIS pixels
var kW = ee.Kernel.fixed(31, 31, W);
var modndviPair_nei0 = modndviPair.map(function(img){
  var img_nei = img.neighborhoodToBands(kW);
  return(img_nei);
});
var modndvisg30 = ee.ImageCollection(modndvi_list);
var modndvisg30_nei = modndvisg30.map(function(img){
  var img_nei = img.neighborhoodToBands(kW);
  return(img_nei);
});

//search Landsat cloudy pixels in the neigborhood of target pixel
var modndviPair_nei_list = modndviPair_nei0.toList(l_size);
var modndviPair_nei_list = list_process.map(function(i){
  i = ee.Number(i);
  var modisone = modndviPair_nei_list.get(i);
  var modisone_img = ee.Image(modisone);
  var maskone = landsatmask_list.get(i);
  var maskone_img = ee.Image(maskone);
  var modisone_masked = modisone_img.updateMask(maskone_img);
  return modisone_masked;});
var modndviPair_nei = ee.ImageCollection(modndviPair_nei_list);

//Calculate the correlation coefficient between similar pixel and the target pixel
var modndviPair_nei_mean = modndviPair_nei.mean();
var landsatNdviPair_mean = landsatNdviPair.mean();

var modndviPair_nei_diffm = modndviPair_nei.map(function(img){
  var img_diff = img.subtract(modndviPair_nei_mean);
  return img_diff;});
var landsatNdviPair_diffm = landsatNdviPair.map(function(img){
  var img_diff = img.subtract(landsatNdviPair_mean);
  return img_diff;});

var modndviPair_nei_diffm_list = modndviPair_nei_diffm.toList(l_size);
var landsatNdviPair_diffm_diffm_list = landsatNdviPair_diffm.toList(l_size);

var modnei_landsat_diff_mul_list = list_process.map(function(i){
  i = ee.Number(i);
  var modnei = modndviPair_nei_diffm_list.get(i);
  var modnei_img = ee.Image(modnei);
  var landsat = landsatNdviPair_diffm_diffm_list.get(i);
  var landsat_img = ee.Image(landsat);
  var landsat_modnei_mul = modnei_img.multiply(landsat_img);
  return landsat_modnei_mul;});

var modnei_landsat_diff_mul = ee.ImageCollection(modnei_landsat_diff_mul_list);
var modnei_landsat_diffmul_sum = modnei_landsat_diff_mul.sum();
var modndviPair_nei_diffm_2 = modndviPair_nei_diffm.map(function(img){
  var img_2 = img.multiply(img);
  return img_2;});
var landsatNdviPair_diffm_2 = landsatNdviPair_diffm.map(function(img){
  var img_2 = img.multiply(img);
  return img_2;});
var modndviPair_nei_diffm2_sum = modndviPair_nei_diffm_2.sum();
var landsatNdviPair_diffm2_sum = landsatNdviPair_diffm_2.sum();
var modnei_landsat_diffmul_sqrt = modndviPair_nei_diffm2_sum.multiply(landsatNdviPair_diffm2_sum)
.sqrt();
var modnei_landsat_corre = modnei_landsat_diffmul_sum.divide(modnei_landsat_diffmul_sqrt);

//Get neighboring similar pixels with the correlation coefficient above 0.8
var threshold_img = ee.Image.constant(0.8);
var threshold_mask = modnei_landsat_corre.gte(threshold_img);
var modnei_landsat_corre_valid = modnei_landsat_corre.multiply(threshold_mask);
var modnei_landsat_corre_sum = modnei_landsat_corre_valid.reduce(ee.Reducer.sum());
var modnei_landsat_corre_normal = modnei_landsat_corre_valid.divide(modnei_landsat_corre_sum);
var modndviPair_match0 = modndviPair_nei0.map(function(img){
  var onemodis_w = img.multiply(modnei_landsat_corre_normal);
  var onemodis_normal = onemodis_w.reduce(ee.Reducer.sum());
  return onemodis_normal;});

var modndvisg30_match0 = modndvisg30_nei.map(function(img){
  var onemodis_w = img.multiply(modnei_landsat_corre_normal);
  var onemodis_normal = onemodis_w.reduce(ee.Reducer.sum());
  return onemodis_normal;});  //generate the reference time series of the target pixel

//For these pixels which may be difficult to generate the reference series, the reference can be obtained by averaging the reference series of the neighboring pixels.
var wm0 = ee.List.repeat(1,21); var wm = ee.List.repeat(wm0,21);  //the size of local window for searching the neighboring pixels
var thresholdmask_sum = threshold_mask.reduce(ee.Reducer.sum());
var thresimg = ee.Image.constant(0);
var valid_flag_mask = thresholdmask_sum.neq(thresimg);
var invalid_flag_mask = thresholdmask_sum.eq(thresimg);
var modndviPair_match = modndviPair_match0.map(function(img){
  var meanimg = img.reduceNeighborhood({reducer: ee.Reducer.mean(),
  kernel:ee.Kernel.fixed(21, 21, wm)});
  var newimg = img.multiply(valid_flag_mask).add(meanimg.multiply(invalid_flag_mask));
  return newimg;});
var modndvisg30_match = modndvisg30_match0.map(function(img){
  var meanimg = img.reduceNeighborhood({reducer: ee.Reducer.mean(),
  kernel:ee.Kernel.fixed(21, 21, wm)});
  var newimg = img.multiply(valid_flag_mask).add(meanimg.multiply(invalid_flag_mask));
  return newimg;});  //search corresponding MODIS time series data of clear Landsat points

//shape correction by linear transfer function
var modndviPair_match_list = modndviPair_match.toList(l_size);
var modndviPair_matchmask_list = list_process.map(function(i){
  i = ee.Number(i);
  var modisone = modndviPair_match_list.get(i);
  var modisone_img = ee.Image(modisone);
  var maskone = landsatmask_list.get(i);
  var maskone_img = ee.Image(maskone);
  var modisone_masked = modisone_img.updateMask(maskone_img);
  return modisone_masked;});

var mergelist = list_process.map(function(i){
    i = ee.Number(i);
    var one = ee.Image(landsatNdviPair_list.get(i)).rename('y');
    var one0 = ee.Image(modndviPair_matchmask_list.get(i)).rename('x');
    var two = one.addBands(one0).addBands(ee.Image.constant(1));
    return two;});
var merge = ee.ImageCollection(mergelist);
var independents = ee.List(['constant','x']);
var dependent = ee.String('y');
var trend = merge.select(independents.add(dependent))
    .reduce(ee.Reducer.linearRegression(independents.length(), 1));
var coefficients = trend.select('coefficients')
.arrayProject([0])
.arrayFlatten([independents]);  //b1 and b0 are solved by minimizing the difference between adjusted time series and the corresponding Landsat time series
var b1 = coefficients.select('x');
var b0 = coefficients.select('constant');

var modndvisg30_adjust = modndvisg30_match.map(function(img){
  var newimg = img.multiply(b1).add(b0);
  return newimg;});

var combine_lm = modndvisg30_adjust.merge(ee.ImageCollection(landsatndvi_list));
var combine_lm_list = combine_lm.toList(l_size.add(m_size));
var modndvisg30_adjust_list = combine_lm_list.slice(0,m_size);
var landsatndvi_list0 = combine_lm_list.slice(m_size,l_size.add(m_size));

//filling the missing values with the corresponding values of modndvisg30_adjust
var list_process = ee.List.sequence(0,m_size.subtract(1));
var syn_series_list = list_process.map(function(i){
  i = ee.Number(i);
  var modimg = ee.Image(modndvisg30_adjust_list.get(i));
  var flag = lm_loc.indexOf(i);
  var imgd = ee.Image(modndvi_list.get(i));
  var landsatimg = ee.Image(landsatndvi_list0.get(flag));
  var landsatmask0 = ee.Image(landsatmask_list.get(flag));
  var validone = ee.Number(1).subtract(list_loc.get(i));
  var validimg = ee.Image.constant(validone);
  var landsatmask = landsatmask0.multiply(validimg);
  var relocimg = landsatimg.multiply(landsatmask).add(modimg
  .subtract(modimg.multiply(landsatmask)));
  return relocimg.rename('MOD_NDVI_INTER')
  .set('system:time_start', imgd.get('system:time_start'));});


var syn_series = ee.ImageCollection(syn_series_list);

//Reduce the residual noise in the synthesized NDVI time series by the weighted SG filter
var list_trend_sgCoeff = ee.List([-0.070588261,-0.011764720,0.038009040,0.078733027,0.11040724,0.13303168,0.14660634,
0.15113123,0.14660634,0.13303168,0.11040724,0.078733027,0.038009040,-0.011764720,-0.070588261]);   //suggested trend parameter:(7,7,0,2)
var list_sg_coeff = ee.List([0.034965038,-0.12820521,0.069930017,0.31468537,0.41724950,0.31468537,
0.069930017,-0.12820521,0.034965038]);   //suggested parameters of SG:(4,4,0,5)
var syn_series_sg = sg_filter.sg_filter_chen(syn_series,list_trend_sgCoeff,list_sg_coeff);

var syn_series_sg_int = syn_series_sg.map(function(img){
  var constant = ee.Image.constant(10000);
  var newimg = img.multiply(constant).toInt16();
  return newimg});

print(syn_series_sg_int);

//output procedure
//choose date range (the whole (0,46) or part of time series (10,15)) to export
var syn_series_sg_int_list = syn_series_sg_int.toList(46);
var syn_series_sg_part_list = syn_series_sg_int_list.slice(0,46); //the whole (0,46) or part of time series (10,15)->(DOY81-113)
var syn_series_sg_part = ee.ImageCollection(syn_series_sg_part_list);
var syn_series_sg_part0 = syn_series_sg_part.toBands();
//rename the bands of output file
var bandname = syn_series_sg_part0.bandNames();
var bandname0 = bandname.map(function(i){
  var namei = ee.String(i);
  var namei0 = ee.String('Landsat_MOD_NDVI_').cat(namei);
  return namei0;
});
syn_series_sg_part0 = syn_series_sg_part0.rename(bandname0);
print(syn_series_sg_part0);

//export data to google drive(set the correct crs)
Export.image.toDrive({
   image:syn_series_sg_part0,
   description: "synseries_sg_anmado",  //the name of output
   region:mod_shp,
   //region:Subarea,  //you can choose the sub area of one scene
   scale:30,
   crs:"EPSG:32652", //UTM zone 52N
   maxPixels:1e13
});

//export to your own asset
/*Export.image.toAsset({
   image:syn_series_sg_part0,
   description: "synseries_CIA",
   assetId:"synseries_sg_CIA",
   region:mod_shp,
   scale:30,
   crs:"EPSG:32655",
   maxPixels:1e13
});*/
